In [1]:
import sys
import requests

In [2]:
from llama_index.core import SummaryIndex, VectorStoreIndex, SimpleDirectoryReader, Settings, Document
from IPython.display import Markdown, display
import os

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

In [4]:
documents = SimpleDirectoryReader("./data").load_data()

In [5]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [6]:
# ollama
llm = Ollama(model="llama3.2", request_timeout=360.0)
Settings.llm = llm

In [7]:
index = VectorStoreIndex(documents)

C:\Users\zhan2\anaconda3\envs\llama\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [8]:
from llama_index.postprocessor.flag_embedding_reranker import (
    FlagEmbeddingReranker,
)

reranker = FlagEmbeddingReranker(
    top_n=5,
    model="BAAI/bge-reranker-large",
)

In [9]:
query_engine = index.as_query_engine(
    similarity_top_k=5,
    node_postprocessors=[reranker],
    response_mode="tree_summarize",  # you can also select other modes like `compact`, `refine`
)

In [10]:
response = query_engine.query("Net earnings in this report")
print(str(response))

$12,475 for the fiscal six months ended July 4, 2021 and $3,626 for the fiscal second quarter ended June 28, 2020.
